In [9]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPool2D
from tensorflow.keras.layers import Concatenate, UpSampling2D, Conv2DTranspose, Cropping2D
from tensorflow.keras.models import Sequential
import numpy as np
import os
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

# Hello
## hello

In [7]:
class LayerUtil:
    def __init__(self):
        pass
    
    def conv(self, filters, name, inputs, kernel_size = (3,3), padding = 'valid', activation = 'relu'):
        return Conv2D(filters = filters, kernel_size = kernel_size, activation = activation, 
                      padding = padding, name = name)(inputs)
    
    def pool(self, name, inputs, pool_size = (2,2), strides = 2):
        return MaxPool2D(pool_size = pool_size, strides = strides, name = name)(inputs)
    
    def deconv(self, filters, name, inputs, kernel_size = (2,2), interpolation='nearest'):
            return Conv2DTranspose(filters = filters, kernel_size = kernel_size, name = name, strides = 2)(inputs)
        
    def crop_and_copy(self, inputs, output, name):
        input_shape = inputs.shape[1:3]
        output_shape = output.shape[1:3]
        
        
        height_crop = int((input_shape[0] - output_shape[0])/2)
        width_crop = int((input_shape[1] - output_shape[1])/2)
        
        crop_layer = Cropping2D((height_crop, width_crop))(inputs)
        concat = Concatenate(axis=3,name = name)([crop_layer, output])
        return concat

class UNet:
    def __init__(self):
#         self.optimizer = optimizer
#         self.loss = loss
#         self.conv_layer = tf.keras.layers.Conv2D()
        self.helper = LayerUtil()
    
    def build_model(self, image_width, image_height, image_channels):
        inputs = Input(shape=(572, 572, 3))
        conv1 = self.helper.conv(64, 'conv1', inputs)
        conv2 = self.helper.conv(64, 'conv2', conv1) #first concat
        pool1 = self.helper.pool('pool1', conv2)

        conv3 = self.helper.conv(128, 'conv3', pool1)
        conv4 = self.helper.conv(128, 'conv4', conv3) #second concat
        pool2 = self.helper.pool('pool2', conv4)
        
        conv5 = self.helper.conv(256, 'conv5', pool2)
        conv6 = self.helper.conv(256, 'conv6', conv5) #third concat
        pool3 = self.helper.pool('pool3', conv6)
        
        conv7 = self.helper.conv(512, 'conv7', pool3)
        conv8 = self.helper.conv(512, 'conv8', conv7) #last concat
        pool4 = self.helper.pool('pool4', conv8)
    
        conv9 = self.helper.conv(1024, 'conv9', pool4)
        conv10 = self.helper.conv(1024, 'conv10', conv9)
        deconv1 = self.helper.deconv(512, 'deconv1', conv10)
        concat1 = self.helper.crop_and_copy(conv8, deconv1, 'concat1')
        
        conv11 = self.helper.conv(512, 'conv11', concat1)
        conv12 = self.helper.conv(512, 'conv12', conv11)
        deconv2 = self.helper.deconv(512, 'deconv2', conv12)
        concat2 = self.helper.crop_and_copy(conv6, deconv2, 'concat2')
        
        conv13 = self.helper.conv(256, 'conv13', concat2)
        conv14 = self.helper.conv(256, 'conv14', conv13)
        deconv3 = self.helper.deconv(256, 'deconv3', conv14)
        concat3 = self.helper.crop_and_copy(conv4, deconv3, 'concat3')
        
        
        conv15 = self.helper.conv(128, 'conv15', concat3)
        conv16 = self.helper.conv(128, 'conv16', conv15)
        deconv4 = self.helper.deconv(128, 'deconv4', conv16)
        concat4 = self.helper.crop_and_copy(conv2, deconv4, 'concat4')
        
        
        conv17 = self.helper.conv(64, 'conv17', concat4)
        conv18 = self.helper.conv(64, 'conv18', conv17)
        
        output = self.helper.conv(2, 'output', conv18, kernel_size = (1,1))
        model = Model(inputs, output)
        
        return model
        
    def compile_model():
        pass
    
    def train_model():
        pass
        

In [3]:
unet = UNet()

In [4]:
model = unet.build_model(572, 572, 3)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 572, 572, 3) 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 570, 570, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 568, 568, 64) 36928       conv1[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 284, 284, 64) 0           conv2[0][0]                      
_______________________________________________________________________________________